In [2]:
import math
print("hello world")

hello world


Here we will read the dataset and transfer it to the base SQL

In [ ]:
file = open("dataset.csv", "r", encoding="utf-8")
output = open("insert_dataset.sql", "w", encoding="utf-8")

# Nombres de columnas en el CSV
columns = [
    "id","product_A_sold_in_the_past","product_B_sold_in_the_past",
    "product_A_recommended","product_A","product_C","product_D",
    "cust_hitrate","cust_interactions","cust_contracts",
    "opp_month","opp_old",
    "competitor_Z","competitor_X","competitor_Y",
    "cust_in_iberia","target_variable"
]

acces = False

for line in file:
    if acces == False:
        acces = True
        continue  # saltar cabecera

    values = line.strip().split(";")

    # Escapar valores de texto con comillas
    escaped = [f"'{v}'" for v in values]

    sql = (
        f"INSERT INTO mi_tabla ({','.join(columns)}) "
        f"VALUES ({','.join(escaped)});\n"
    )

    output.write(sql)

file.close()
output.close()

print("Archivo SQL generado.")


Archivo SQL generado.


In [11]:
pip install sklearn


Note: you may need to restart the kernel to use updated packages.Collecting sklearn
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'



  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag

In [10]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.Collecting scikit-learn
     ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
     - -------------------------------------- 0.2/8.9 MB 4.6 MB/s eta 0:00:02
     ------ --------------------------------- 1.4/8.9 MB 15.0 MB/s eta 0:00:01
     ----------- ---------------------------- 2.5/8.9 MB 17.8 MB/s eta 0:00:01
     ------------- -------------------------- 3.1/8.9 MB 16.4 MB/s eta 0:00:01
     ------------------ --------------------- 4.0/8.9 MB 18.4 MB/s eta 0:00:01
     ------------------------- -------------- 5.6/8.9 MB 18.8 MB/s eta 0:00:01
     ------------------------------ --------- 6.8/8.9 MB 19.9 MB/s eta 0:00:01
     ------------------------------------ --- 8.2/8.9 MB 21.0 MB/s eta 0:00:01
     ---------------------------------------  8.9/8.9 MB 20.3 MB/s eta 0:00:01
     ---------------------------------------- 8.9/8.9 MB 18.9 MB/s eta 0:00:00




[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier  # o RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error

# 1. Cargar CSV
df = pd.read_csv("dataset.csv", sep=";")

# 2. Separar variables de entrada y variable objetivo
X = df.drop("target_variable", axis=1)
y = df["target_variable"]

# Convertir columnas no numéricas si existen
X = pd.get_dummies(X)

# 3. Dividir en train / test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Entrenar modelo (clasificación)
model = RandomForestClassifier()   # si es regresión: RandomForestRegressor()
model.fit(X_train, y_train)

# 5. Evaluar
pred = model.predict(X_test)

print("Exactitud:", accuracy_score(y_test, pred))
# En caso de regresión: print("MSE:", mean_squared_error(y_test, pred))

# 6. Guardar el modelo para usarlo luego
import joblib
joblib.dump(model, "modelo_entrenado.pkl")

print("Modelo entrenado y guardado.")


Exactitud: 0.8238161559888579
Modelo entrenado y guardado.


In [ ]:
import joblib
import pandas as pd

model = joblib.load("modelo_entrenado.pkl")

nuevo = {
    "product_A_sold_in_the_past": 1,
    "product_B_sold_in_the_past": 0,
    "product_A_recommended": 1,
    "product_A": 1,
    "product_C": 0,
    "product_D": 1,
    "cust_hitrate": 0.78,
    "cust_interactions": 12,
    "cust_contracts": 2,
    "opp_month": 3,
    "opp_old": 0,
    "competitor_Z": 0,
    "competitor_X": 0,
    "competitor_Y": 1,
    "cust_in_iberia": 1
}

nuevo_df = pd.DataFrame([nuevo])
nuevo_df = pd.get_dummies(nuevo_df)

# Asegurar que las columnas coinciden
for col in model.feature_names_in_:
    if col not in nuevo_df.columns:
        nuevo_df[col] = 0

pred = model.predict(nuevo_df[model.feature_names_in_])

print("Valoración o predicción:", pred[0])


Valoración o predicción: 0
